In [2]:
import cv2 as cv
import numpy as np
from datetime import datetime, timedelta
import os
import tensorflow as tf
import pandas as pd

def is_valid(now, time_end, frame, save_path, curr_time):
    if now >= time_end:
        filename = f"{curr_time.replace(':', '-')}.png"
        cv.imwrite(os.path.join(save_path, filename), frame)
        return False
    return True

def preprocess_frame(frame, target_size=(200, 200)):
    """Resize and normalize the frame for model prediction."""
    resized_frame = cv.resize(frame, target_size)
    return np.expand_dims(resized_frame / 255.0, axis=0)

def draw_info(frame, center, curr_date, curr_time, predicted_name, confidence_score):
    """Draw date, time, and predicted name on the frame."""
    (text_width, _), _ = cv.getTextSize(curr_date, cv.FONT_HERSHEY_TRIPLEX, 0.75, 1)
    text_x = center[0] - (text_width // 2)
    text_y = center[1] + 20  # Adjust vertical position

    cv.putText(frame, curr_date, (text_x - 200, text_y - 200), 
               cv.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)
    cv.putText(frame, curr_time, (text_x - 200, text_y - 185), 
               cv.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)
    cv.putText(frame, f"Predicted: {predicted_name}", (text_x - 200, text_y - 170), 
               cv.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)
    cv.putText(frame, f"Confidence: {confidence_score*100:.2f}%", (text_x - 200, text_y - 155),
               cv.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)

def log_final_prediction_to_excel(filepath, date, time, name, confidence):
    # Prepare data to log only the final prediction
    df_entry = pd.DataFrame([{
        "Date": date,
        "Time": time,
        "Prediction": name,
        "Confidence (%)": round(confidence * 100, 2)
    }])

    # Write the data to an Excel file (if it doesn't exist, create it)
    df_entry.to_excel(filepath, index=False, header=True)

def main():
    # Load the trained model
    try:
        model = tf.keras.models.load_model('recognition_model.h5')
    except OSError as e:
        print(f"Error: {e}")
        print("Ensure the model file 'recognition_model.h5' exists in the correct path.")
        return

    # Load class names
    try:
        with open('class_names.txt', 'r') as f:
            class_names = [line.strip() for line in f.readlines()]
    except FileNotFoundError as e:
        print(f"Error: {e}")
        print("Ensure the file 'class_names.txt' exists in the correct path.")
        return

    print("Class Names:", class_names)
    print('\nCamera ON!!')

    # Initialize camera
    cam = cv.VideoCapture(0)
    if not cam.isOpened():
        print("Error: Could not open camera.")
        return

    # Set up timing
    time_start = datetime.now()
    time_end = time_start + timedelta(seconds=6)

    # Create a directory to save captured frames
    save_directory = os.path.join("D:", "Entries", str(datetime.today().date()))
    os.makedirs(save_directory, exist_ok=True)

    # Excel log file path
    excel_path = os.path.join(save_directory, "attendance_log.xlsx")

    # Precompute mask
    ret, frame = cam.read()
    if not ret:
        print("Error: Failed to capture initial frame.")
        return

    height, width, _ = frame.shape
    center = (width // 2, height // 2)
    mask = cv.rectangle(np.zeros((height, width), dtype='uint8'), 
                        (center[0] - 270, center[1] - 150), 
                        (center[0] + 270, center[1] + 150), 
                        255, thickness=-1)

    final_prediction = None
    final_prediction_confidence = None
    final_date = None
    final_time = None

    while True:
        ret, frame = cam.read()
        if not ret:
            print("Error: Failed to capture frame.")
            break

        frame = cv.flip(frame, 1)

        now = datetime.now()
        curr_date = now.strftime('%d-%m-%y')
        curr_time = now.strftime('%H-%M-%S')

        if not is_valid(now, time_end, frame, save_directory, curr_time):
            break

        masked_vid = cv.bitwise_and(frame, frame, mask=mask)

        cv.rectangle(masked_vid, 
                     (center[0] - 275, center[1] - 155), 
                     (center[0] + 275, center[1] + 155), 
                     (255, 255, 255), thickness=1)

        # Optionally crop the region inside the mask for better predictions
        roi = frame[center[1]-150:center[1]+150, center[0]-270:center[0]+270]
        input_frame = preprocess_frame(roi)

        predictions = model.predict(input_frame)
        print("Predictions probabilities:", predictions)

        predicted_label = np.argmax(predictions)
        confidence_score = np.max(predictions)

        CONFIDENCE_THRESHOLD = 0.4

        if confidence_score >= CONFIDENCE_THRESHOLD and predicted_label < len(class_names):
            predicted_name = class_names[predicted_label]
        else:
            predicted_name = "Not Identified, Try Again"

        print(f"Predicted: {predicted_name}, Confidence: {confidence_score:.2f}")

        # Save the final prediction and confidence
        final_prediction = predicted_name
        final_prediction_confidence = confidence_score
        final_date = curr_date
        final_time = curr_time

        draw_info(masked_vid, center, curr_date, curr_time, predicted_name, confidence_score)

        cv.imshow('Masked Video', masked_vid)

        if cv.waitKey(1) & 0xFF == ord('q'):
            break

    # After the loop, log the final prediction
    if final_prediction is not None and final_prediction_confidence is not None:
        log_final_prediction_to_excel(excel_path, final_date, final_time, final_prediction, final_prediction_confidence)

    cam.release()
    cv.destroyAllWindows()

if __name__ == "__main__":
    main()


Class Names: ['.ipynb_checkpoints', 'chris_evans', 'chris_hemsworth', 'mark_ruffalo', 'robert_downey_jr', 'scarlett_johansson']

Camera ON!!
1/1 [==============================] - 0s 191ms/step
Predictions probabilities: [[2.58176122e-04 5.99715859e-03 1.22530766e-01 3.02433223e-02
  8.39312673e-01 1.65796862e-03]]
Predicted: robert_downey_jr, Confidence: 0.84
1/1 [==============================] - 0s 56ms/step
Predictions probabilities: [[3.0323034e-04 7.2173057e-03 1.2760009e-01 3.2930065e-02 8.3005083e-01
  1.8985695e-03]]
Predicted: robert_downey_jr, Confidence: 0.83
1/1 [==============================] - 0s 44ms/step
Predictions probabilities: [[3.4851555e-04 7.9579363e-03 1.2819014e-01 3.8736034e-02 8.2278126e-01
  1.9861893e-03]]
Predicted: robert_downey_jr, Confidence: 0.82
1/1 [==============================] - 0s 66ms/step
Predictions probabilities: [[3.3322515e-04 7.7268607e-03 1.1809749e-01 3.7891492e-02 8.3405513e-01
  1.8959284e-03]]
Predicted: robert_downey_jr, Confidenc